In [1]:
from IPython.display import Image

# Algorithme COordinate Rotation DIgital Computer

Les premières machines et calcultaeurs ne pouvaient executer qu'uniquement des additions et soustractions. Le problème est que ces opérations ne suffisent pas à la plupart des scientifiques. Ainsi l'algorithme CORDIC a été crée. Il permet le calcul de fonctions trigonométriques en utilisant uniquement des additions et soustractions. Cela relève d'une prouesse. On se propose donc de découvrir et d'implémenter ce bel algorithme.

## Quelques bases

Avant tout, il faut comprendre les bases sur les différentes fonctions sinus, cosinus et tangeante. Ceci n'est pas un cours de maths de 3ème.


On peut facilement visualiser à quoi respondent ces fonctions en utilisant le cercle trigonométrique : 

In [2]:
Image(url="https://lecluseo.scenari-community.org/2DE/Trigonometrie/res/cercletrigo.png")

** à noter ** : on définit la tangeante par : $ \tan(x)= \frac {\sin x}{\cos x}$. On remarquera que le domaine de définition de $\tan$ est limité par sa définition sur $]-k \frac \pi 2, k \frac \pi 2[, k \in \mathbb Z$

Comme écrit sur ce dessin, on lit la valeur du cosinus d'une valeur sur l'axe des abscisses et le sinus sur l'oronnée.

Prenons un exemple : $sin(45°)=sin(\frac  \pi 4)= \frac {\sqrt 2} 2$

De là, nous pouvons facilement donner plusieurs formules qui découlent des propriétées de symétrie des fonctions cosinus et sinus :